In [27]:
import networkx as nx
import pandas as pd

data = pd.read_csv('C:/Users/timothy/Downloads/Camden_Licensing_Applications_Beta.csv',parse_dates=['Received Date'])
gaming_license = data[data['Application Type']=='Gaming Notification'].copy()

In [51]:
import numpy as np
#gaming_license['path'] = gaming_license['Received Date'].dt.year - 2007
gaming_license['path'] = 1
gaming_license['Origin-Destination'] = np.where(gaming_license['Received Date']==min(gaming_license['Received Date']),'Origin','Destination')
gaming_license['PathId'] = 'origin' + gaming_license['Application Reference'].astype(str) #np.where(gaming_license['Received Date'].dt.year == 2007,'2008_2007',
                                   # gaming_license['Received Date'].dt.year.astype(str)+str('_2007'))


In [52]:
gaming_license.to_csv('gaming_with_path.csv')

In [152]:
origin_destination = data[data['Application Type']=='Gaming Notification'].copy()

geo_loc = origin_destination.groupby(['Premises Name']).min()

origin_destination['Premises Name'].value_counts()
spread = pd.DataFrame(columns=['Origin-Destination','Premise','pathid','latitude','longitude','Received Date'])
for year in origin_destination['Received Date'].dt.year.unique():
    table = origin_destination[origin_destination['Received Date'].dt.year==year]
    original = table['Premises Name'][table['Received Date']== min(table['Received Date'])].iloc[0]
    for destination in table['Premises Name'].unique():
        spread = spread.append({'Origin-Destination':'Destination','Premise':destination,
                                'pathid':str(year)+"_"+str(destination)+"_"+str(original),
                      'latitude':min(table['Latitude'][table['Premises Name']==destination]),
                       'longitude':min(table['Longitude'][table['Premises Name']==destination]),
                               'Received Date':min(table['Received Date'][table['Premises Name']==destination])},ignore_index=True)
        spread = spread.append({'Origin-Destination':'Origin','Premise':original,
                                'pathid':str(year)+"_"+str(destination)+"_"+str(original),
                      'latitude':np.mean(table['Latitude'][table['Premises Name']==original]),
                       'longitude':np.mean(table['Longitude'][table['Premises Name']==original]),
                      'Received Date':min(table['Received Date'][table['Premises Name']==original])},ignore_index=True)
    
spread

,Origin-Destination,Premise,pathid,latitude,longitude,Received Date
0,Destination,Secrets (Eversholt St),2017_Secrets (Eversholt St)_Clockhouse,51.529302,-0.132585,2017-07-04
1,Origin,Clockhouse,2017_Secrets (Eversholt St)_Clockhouse,51.521044,-0.109235,2017-01-24
2,Destination,Clockhouse,2017_Clockhouse_Clockhouse,51.521044,-0.109235,2017-01-24
3,Origin,Clockhouse,2017_Clockhouse_Clockhouse,51.521044,-0.109235,2017-01-24
4,Destination,Oxford Arms,2007_Oxford Arms_Good Mixer,51.540542,-0.143662,2007-11-20
5,Origin,Good Mixer,2007_Oxford Arms_Good Mixer,51.540039,-0.144497,2007-03-12
6,Destination,Smugglers Tavern,2007_Smugglers Tavern_Good Mixer,51.523545,-0.140725,2007-11-21
7,Origin,Good Mixer,2007_Smugglers Tavern_Good Mixer,51.540039,-0.144497,2007-03-12
8,Destination,Golden Lion,2007_Golden Lion_Good Mixer,51.539006,-0.135735,2007-12-17
9,Origin,Good Mixer,2007_Golden Lion_Good Mixer,51.540039,-0.144497,2007-03-12


In [153]:
spread.to_csv('paths_gaming.csv')

In [ ]:

for destination in origin_destination['Premises Name'].unique():
    spread = spread.append({'Origin-Destination':'Destination','Premise':destination,
                                'pathid':str(str(destination)+"_"+str(original),
                      'latitude':min(origin_destination['Latitude'][table['Premises Name']==destination]),
                       'longitude':min(origin_destination['Longitude'][table['Premises Name']==destination])},ignore_index=True)
    spread = spread.append({'Origin-Destination':'Origin','Premise':original,
                                'pathid':str(destination)+"_"+str(original),
                      'latitude':np.mean(origin_destination['Latitude'][origin_destination['Premises Name']==original]),
                       'longitude':np.mean(origin_destination['Longitude'][origin_destination['Premises Name']==original])},ignore_index=True)
    
    

In [148]:
propegation = pd.DataFrame(columns=['Premise','pathid','order of points','latitude','longitude','Received Date'])

work_table = origin_destination.groupby('Premises Name').min()
pathcount = 0
while len(work_table.index)>1:
    min_time = min(work_table['Received Date'])
    min_index = work_table[work_table['Received Date']==min_time].index[0]
    work_table.loc[min_index]
    propegation = propegation.append({'Premise':min_index,'pathid':work_table.loc[min_index]['Ward Name'],'order of points':pathcount,
                        'latitude':work_table.loc[min_index]['Latitude'],
                       'longitude':work_table.loc[min_index]['Longitude'],
                        'Received Date':work_table.loc[min_index]['Received Date']},ignore_index=True)
    work_table = work_table.drop(min_index)
    pathcount+=1
    
propegation.to_csv('propegation.csv')
propegation

,Premise,pathid,order of points,latitude,longitude,Received Date
0,Good Mixer,Camden Town with Primrose Hill,0.0,51.540039,-0.144497,2007-03-12
1,Neptune,NaN,1.0,NaN,NaN,2007-07-11
2,Somerstown Coffee House,St Pancras and Somers Town,2.0,51.529990,-0.130767,2007-07-11
3,Spearmint Rhino,Bloomsbury,3.0,51.523398,-0.136463,2007-07-12
4,Lyttelton Arms,Regent's Park,4.0,51.534911,-0.139649,2007-08-11
5,Prince Of Wales' Feathers,Bloomsbury,5.0,51.523937,-0.138173,2007-08-11
6,Old Bell,Kilburn,6.0,51.537207,-0.192154,2007-08-15
7,The Crown & Anchor,Regent's Park,7.0,51.526920,-0.136481,2007-09-25
8,Odeon Cinema,Camden Town with Primrose Hill,8.0,51.514280,-0.143695,2007-09-27
9,Sir Robert Peel,Gospel Oak,9.0,51.549575,-0.152850,2007-10-25
